# Environmental Assessment


In [ ]:
#!pip install scattertext wordcloud spacy

#import sys
#print(sys.executable)

/opt/anaconda3/envs/textanalysis/bin/python


In [76]:
import os
from pathlib import Path

import pandas as pd
import spacy
import scattertext as st
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Remove width limit (show full content)
pd.set_option('display.max_colwidth', None)

# Or set a specific width (e.g., 500 characters)
pd.set_option('display.max_colwidth', 500)
pd.set_option("display.max_rows", None)

# Get the directory where this notebook is located
#notebook_dir = Path.cwd()  # or os.getcwd()
#print(f"Current directory: {notebook_dir}")

# Or force it to be the notebook's directory
# notebook_dir = Path(__file__).parent  # doesn't work in notebooks

In [ ]:
#%pip install pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.0/36.0 MB 43.7 MB/s  0:00:00 eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Verify the current working directory
notebook_dir = Path.cwd()  # or os.getcwd()
print(f"Current directory: {notebook_dir}")

Current directory: /Users/Dora/git/consulting/nepa/code


In [3]:
import pandas as pd

# Read entire file
projects = pd.read_parquet('../data/processed/ea/projects.parquet')
process = pd.read_parquet("../data/processed/ea/processes.parquet")
documents = pd.read_parquet('../data/processed/ea/documents.parquet')
pages = pd.read_parquet('../data/processed/ea/pages.parquet')

## Projects 

### Project type

Use this feature to determine clean vs fossil energy of each project. The difficulty here will be that many projects have more than one classification -- e.g.,  "Utilities (electricity, gas, telecommunication), Broadband"

In [ ]:
# view unqiue project type values to see how they match with clean energy criteria
projects["project_type"].explode().unique()

array(['Utilities (electricity, gas, telecommunications)', 'Broadband',
       'Conventional Energy Production - Land-based Oil & Gas', 'Pipelines', 'Routine Maintenance',
       'Mining - Non-Metallic Minerals', 'Public and Recreational Land Use', 'Ecosystem Management and Restoration',
       'Vegetation and Fuels Management', 'Threatened and Endangered Species Management',
       'Water Resources - Irrigation and Water Supply', 'Agriculture', 'Rangeland Management', 'Electricity Transmission',
       'Surface Transportation - Other', 'Land Use or Forest Management Plan', 'Waste Management', 'Mining - Metals',
       'Water Resources - Other', 'Conventional Energy Production - Coal', 'Laws, Policies, Regulations, and Guidance',
       'Land Development - Other', 'Ports and Waterways', 'Research and Development',
       'Renewable Energy Production - Hydropower', 'Habitat Conservation Plan', 'Renewable Energy Production - Other',
       'Manufacturing', 'Renewable Energy Production - 

In [ ]:
# define clean energy types
clean_energy_types = {
    "Carbon Capture and Sequestration",
    "Conventional Energy Production - Nuclear",
    "Renewable Energy Production - Biomass",
    "Renewable Energy Production - Energy Storage",
    "Renewable Energy Production - Geothermal",
    "Renewable Energy Production - Hydrokinetic",
    "Renewable Energy Production - Hydropower",
    "Renewable Energy Production - Solar",
    "Renewable Energy Production - Wind, Offshore",
    "Renewable Energy Production - Wind, Onshore",
    "Renewable Energy Production - Other",
    "Nuclear Technology",
    "Electricity Transmission",
    "Utilities (electricity, gas, telecommunications)"
}


In [83]:
# apply clean energy dictionary to create new "clean energy" column
import numpy as np

projects["clean_energy"] = (
    projects["project_type"]
    .apply(lambda x: any(t in clean_energy_types for t in x) if isinstance(x, (list, np.ndarray)) else False)
)

projects.head()

,project_id,project_title,project_sector,project_type,project_description,project_sponsor,project_location,clean_energy
0,{'value': '6e9fc6608e30977c74305d2a98628a13'},Coldfoot Cell Tower,[Transportation and Infrastructure],"[Utilities (electricity, gas, telecommunications), Broadband]",,[GCI],"[Coldfoot, Yukon-Koyukuk Census Area, AK (Lat/Lon: 67.2500, -150.1750)]",True
1,{'value': 'e1f95fc5d710fea657ab131cb5b63f9c'},CRC Facility Re-rout for Lehnhardt Wells,"[Energy Production and Management, Miscellaneous and Emerging Technologies, Transportation and Infrastructure]","[Conventional Energy Production - Land-based Oil & Gas, Pipelines, Routine Maintenance]",,[California Resources Corp. (CRC)],"[Kern Front Oil Field, Kern County, CA (Lat/Lon: 35.48, -119.05)]",False
2,{'value': 'a48fc2e9268e31474c3ef9a1a40d2d5d'},Balboa Well Abandonment 60-24B,[Energy Production and Management],[Conventional Energy Production - Land-based Oil & Gas],,"[California Resources of Elk Hills, LLC]","[Buena Vista Hills Oil Field, Kern County, CA (Lat/Lon: 35.2000, -119.4500)]",False
3,{'value': '39d059404366825eda2052316c81c34a'},Three Lost Hills Well Abandonments,[Energy Production and Management],[Conventional Energy Production - Land-based Oil & Gas],,[Chevron USA Inc.],"[Lost Hills, Kern County, CA (Lat/Lon: 35.6069, -119.6604)]",False
4,{'value': 'e36ab077e8323fbb6f67fb76b47c5820'},Ehrenberg Wash Material Source Competitive Mineral Material Sale,[Materials and Manufacturing],[Mining - Non-Metallic Minerals],,"[Fisher Sand & Gravel, Inc.]","[Ehrenberg, La Paz County, Arizona (Lat/Lon: 33.58, -114.45)]",False


In [73]:
projects.shape

(3083, 8)

In [84]:
# how many were flagged?  860 were flagged for environmental review
projects["clean_energy"].value_counts(dropna=False)


clean_energy
False    2223
True      860
Name: count, dtype: int64

In [ ]:
(860/3083) * 100  # 28% of environmental review is for

27.894907557573788

In [90]:
# view project title and type of clean energy projects under environmental review
#clean_energy = projects.loc[projects["clean_energy"].notna(), ["project_title", "project_type", "clean_energy"]]
clean_energy = projects[projects["clean_energy"] == True][["project_title", "project_type", "clean_energy"]]

clean_energy.head(20)

# save 
clean_energy.to_csv("../output/projects_clean_energy.csv", index=False)


In [89]:
clean_energy.head(5)

,project_title,project_type,clean_energy
0,Coldfoot Cell Tower,"[Utilities (electricity, gas, telecommunications), Broadband]",True
9,Golden Valley 230 kV Transmission Line Project,[Electricity Transmission],True
50,Holmes Western (HW) Buena Vista Hills; 34 APD’s for Leutholtz,"[Conventional Energy Production - Land-based Oil & Gas, Pipelines, Electricity Transmission, Waste Management, Threatened and Endangered Species Management, Ecosystem Management and Restoration]",True
61,Rock Creek Hydroelectric Project,"[Renewable Energy Production - Hydropower, Electricity Transmission, Water Resources - Irrigation and Water Supply]",True
69,Lugo-Victorville Remedial Action Scheme Project,"[Electricity Transmission, Utilities (electricity, gas, telecommunications), Renewable Energy Production - Other]",True


In [75]:

utilities_label = "Utilities (electricity, gas, telecommunications)"

projects.loc[
    (projects["clean_energy"].notna()) &
    (projects["project_type"].apply(
        lambda x: utilities_label in x if isinstance(x, (list, np.ndarray)) else False
    )),
    ["project_title", "project_type", "clean_energy"]
].head(100)


,project_title,project_type,clean_energy
0,Coldfoot Cell Tower,"[Utilities (electricity, gas, telecommunications), Broadband]",True
69,Lugo-Victorville Remedial Action Scheme Project,"[Electricity Transmission, Utilities (electricity, gas, telecommunications), Renewable Energy Production - Other]",True
70,"CVTC – Tok Cutoff, South Slana, Existing and New Fiber Lines","[Utilities (electricity, gas, telecommunications), Broadband]",True
82,Cahuilla Ranger Station Reconstruction Project,"[Land Development - Other, Public and Recreational Land Use, Utilities (electricity, gas, telecommunications), Waste Management, Mining - Non-Metallic Minerals]",True
85,Lost Hills; 13 Applications for Permit to Drill,"[Conventional Energy Production - Land-based Oil & Gas, Pipelines, Utilities (electricity, gas, telecommunications), Waste Management, Threatened and Endangered Species Management, Ecosystem Management and Restoration, Surface Transportation - Other]",True
...,...,...,...
1114,White Tanks/Miller Road Temporary Land Use Closure,"[Public and Recreational Land Use, Utilities (electricity, gas, telecommunications), Laws, Policies, Regulations, and Guidance, Land Use or Forest Management Plan]",True
1115,Buena Vista Hills; Two APD’s for 624H-6D & 658H-36-6D,"[Conventional Energy Production - Land-based Oil & Gas, Pipelines, Utilities (electricity, gas, telecommunications), Waste Management, Threatened and Endangered Species Management, Ecosystem Management and Restoration]",True
1117,Andrews Farms Right-of-Way,"[Surface Transportation - Other, Utilities (electricity, gas, telecommunications), Routine Maintenance]",True
1138,Rocky Mountain Natural Gas Telluride Leg-Off,"[Pipelines, Routine Maintenance, Utilities (electricity, gas, telecommunications)]",True


### Project location


In [41]:
#type(projects.project_sector)

#projects.project_sector.tolist()
#projects.project_location.tolist()
projects.project_location.head(500)


0           [Coldfoot, Yukon-Koyukuk Census Area, AK (Lat/Lon: 67.2500, -150.1750)]
1                 [Kern Front Oil Field, Kern County, CA (Lat/Lon: 35.48, -119.05)]
2      [Buena Vista Hills Oil Field, Kern County, CA (Lat/Lon: 35.2000, -119.4500)]
3                       [Lost Hills, Kern County, CA (Lat/Lon: 35.6069, -119.6604)]
4                     [Ehrenberg, La Paz County, Arizona (Lat/Lon: 33.58, -114.45)]
                                           ...                                     
495                             [Hardee County, Florida (Lat/Lon: 27.617, -81.775)]
496                     [Lost Hills, Kern County, CA (Lat/Lon: 35.6000, -119.6600)]
497                             [Adams County, Idaho (Lat/Lon: 44.8833, -116.4500)]
498                                 [Lake County, CA (Lat/Lon: 39.0907, -122.6438)]
499                  [Maricopa County, Yuma County, AZ (Lat/Lon: 33.088, -113.752)]
Name: project_location, Length: 500, dtype: object

In [42]:
process.head()

,project_id,process_family,process_type,lead_agency
0,{'value': '6e9fc6608e30977c74305d2a98628a13'},National Environmental Policy Act (NEPA) Review,Environmental Assessment (EA),[Department of the Interior - Bureau of Land Management]
1,{'value': 'e1f95fc5d710fea657ab131cb5b63f9c'},National Environmental Policy Act (NEPA) Review,Environmental Assessment (EA),[Department of the Interior - Bureau of Land Management]
2,{'value': 'a48fc2e9268e31474c3ef9a1a40d2d5d'},National Environmental Policy Act (NEPA) Review,Environmental Assessment (EA),[Department of the Interior - Bureau of Land Management]
3,{'value': '39d059404366825eda2052316c81c34a'},National Environmental Policy Act (NEPA) Review,Environmental Assessment (EA),[Department of the Interior - Bureau of Land Management]
4,{'value': 'e36ab077e8323fbb6f67fb76b47c5820'},National Environmental Policy Act (NEPA) Review,Environmental Assessment (EA),[Department of the Interior - Bureau of Land Management]


In [91]:
process.process_type.unique()

array(['Environmental Assessment (EA)'], dtype=object)

In [103]:
process["lead_agency"].explode().value_counts()

lead_agency
Department of the Interior - Bureau of Land Management                                       2217
Department of Energy - Department of Energy                                                   494
Department of Energy - Power Marketing Administration                                         140
Department of Energy - National Nuclear Security Administration                                53
Department of Agriculture - Forest Service                                                     51
Department of Energy - Energy Programs                                                         38
Major Independent Agencies - Corps of Engineers--Civil Works                                   14
                                                                                               12
Department of the Interior - Bureau of Reclamation                                             11
Department of the Interior - Department of the Interior                                         8
Departme

In [ ]:

utilities_label = "Utilities (electricity, gas, telecommunications)"

projects.loc[
    (projects["clean_energy"].notna()) &
    (projects["project_type"].apply(
        lambda x: utilities_label in x if isinstance(x, (list, np.ndarray)) else False
    )),
    ["project_title", "project_type", "clean_energy"]
].head(100)


,project_title,project_type,clean_energy
0,Coldfoot Cell Tower,"[Utilities (electricity, gas, telecommunications), Broadband]",True
69,Lugo-Victorville Remedial Action Scheme Project,"[Electricity Transmission, Utilities (electricity, gas, telecommunications), Renewable Energy Production - Other]",True
70,"CVTC – Tok Cutoff, South Slana, Existing and New Fiber Lines","[Utilities (electricity, gas, telecommunications), Broadband]",True
82,Cahuilla Ranger Station Reconstruction Project,"[Land Development - Other, Public and Recreational Land Use, Utilities (electricity, gas, telecommunications), Waste Management, Mining - Non-Metallic Minerals]",True
85,Lost Hills; 13 Applications for Permit to Drill,"[Conventional Energy Production - Land-based Oil & Gas, Pipelines, Utilities (electricity, gas, telecommunications), Waste Management, Threatened and Endangered Species Management, Ecosystem Management and Restoration, Surface Transportation - Other]",True
...,...,...,...
1114,White Tanks/Miller Road Temporary Land Use Closure,"[Public and Recreational Land Use, Utilities (electricity, gas, telecommunications), Laws, Policies, Regulations, and Guidance, Land Use or Forest Management Plan]",True
1115,Buena Vista Hills; Two APD’s for 624H-6D & 658H-36-6D,"[Conventional Energy Production - Land-based Oil & Gas, Pipelines, Utilities (electricity, gas, telecommunications), Waste Management, Threatened and Endangered Species Management, Ecosystem Management and Restoration]",True
1117,Andrews Farms Right-of-Way,"[Surface Transportation - Other, Utilities (electricity, gas, telecommunications), Routine Maintenance]",True
1138,Rocky Mountain Natural Gas Telluride Leg-Off,"[Pipelines, Routine Maintenance, Utilities (electricity, gas, telecommunications)]",True


In [44]:
documents.head()

,project_id,document_id,document_type,document_title,prepared_by,ce_category,file_id,file_name,total_pages,main_document
0,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES
1,{'value': '6e9fc6608e30977c74305d2a98628a13'},18ba4bebd27753266190e6a95b6890dc,,,[],,18ba4bebd27753266190e6a95b6890dc,ARF_PDF.pdf,3,NO
2,{'value': '6e9fc6608e30977c74305d2a98628a13'},ab609d98a24655188c809403825bf9e6,,,[],,ab609d98a24655188c809403825bf9e6,FONSI_PDF.pdf,1,NO
3,{'value': '6e9fc6608e30977c74305d2a98628a13'},299fe27f5e25bda53e0a1e4883a76c61,EA,Coldfoot Cell Tower,[Department of the Interior - Bureau of Land Management],,299fe27f5e25bda53e0a1e4883a76c61,DR_PDF.pdf,7,YES
4,{'value': 'e1f95fc5d710fea657ab131cb5b63f9c'},fb1bfb42542b584db11892c34f9be138,,,[],,fb1bfb42542b584db11892c34f9be138,DR_CO60-2018-0021-EA_-_CRC_-_Facility_Re-rout_for_Lenhardt_Wells.pdf,2,YES


In [45]:
pages.head()

,document_id,page_number,page_text
0,2d9415b3c9309d9670f09501ca618ce3,1,"Coldfoot Cell Tower, F-97066 \n \nPage 1 \n \nEnvironmental Assessment \nGCI Coldfoot Cell Tower \nDOI-BLM-AK-2016-F030-0009-EA \n \n \n \n \n \n \n \nBureau of Land Management \nCentral Yukon Field Office \n1150 University Avenue \nFairbanks, Alaska 99709 \n \nFebruary 2016 \n \n"
1,2d9415b3c9309d9670f09501ca618ce3,2,"Coldfoot Cell Tower, F-97066 \n \nPage 2 \n \n1. INTRODUCTION \n \nIDENTIFYING INFORMATION \n \nTitle, EA Number, and Type of Project: \n \nDOI-BLM-AK-F030-2016-0009-EA; Communication Site \n \nLocation of Proposed Action: \n \nThe proposed tower center coordinates are 67° 15' 16.35"" N, 150° 11' 05.99"" W (NAD 83) near \nthe Coldfoot Airport, mile post 175 off the Dalton Highway more particularly described as \nwithin Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, Alaska. \n \nName and Loca..."
2,2d9415b3c9309d9670f09501ca618ce3,3,"Coldfoot Cell Tower, F-97066 \n \n \n \n \n \nPage 3 \n \n \n \nFigure 1. Proposed GCI Wireless facility location Aerial photograph of the Coldfoot airport, access road and general area.\n"
3,2d9415b3c9309d9670f09501ca618ce3,4,"Coldfoot Cell Tower, F-97066 \n \nPage 4 \n \n2. PROPOSED ACTION AND ALTERNATIVES \nDESCRIPTION OF THE PROPOSED ACTION \nGCI is proposing to install a communication tower site on public lands near the Coldfoot airport, \nmile post 175 off the Dalton Highway within Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, \ncontaining 2.29 acre. They propose a 20 year authorization with the option of renewal to \nconstruct, operate, maintain, and decommission. \n \nThe facility would provide UMTS (3G..."
4,2d9415b3c9309d9670f09501ca618ce3,5,"Coldfoot Cell Tower, F-97066 \n \nPage 5 \n \nDESCRIPTION OF ALTERNATIVES ANALYZED IN DETAIL \n \nNo Action Alternative \n \nUnder the No Action Alternative, BLM would not grant the lease. Currently there is no wireless \nvoice and data coverage in Coldfoot, and that would not change under the No Action Alternative. \n \nALTERNATIVES CONSIDERED BUT NOT ANALYZED IN DETAIL \n \nGCI considered the collocation of its wireless facilities on the Summit Telephone tower located \nwithin the communi..."


In [46]:
print(pages.page_text[1])

Coldfoot Cell Tower, F-97066 
 
Page 2 
 
1. INTRODUCTION 
 
IDENTIFYING INFORMATION 
 
Title, EA Number, and Type of Project: 
 
DOI-BLM-AK-F030-2016-0009-EA; Communication Site 
 
Location of Proposed Action: 
 
The proposed tower center coordinates are 67° 15' 16.35" N, 150° 11' 05.99" W (NAD 83) near 
the Coldfoot Airport, mile post 175 off the Dalton Highway more particularly described as 
within Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, Alaska. 
 
Name and Location of Preparing Office: 
 
Central Yukon Field Office, 1150 University Avenue, Fairbanks, Alaska 99709 
 
Identify the case file number:  F-97066 
 
Applicant Name:  GCI Communication Corporation (GCI) 
 
PURPOSE AND NEED FOR ACTION 
 
GCI is proposing to install a communication tower site on public lands near the Coldfoot Airport, 
mile post 175 off the Dalton Highway, Alaska. The need for action is established by the Bureau 
of Land Management’s (BLM) responsibility under Title V of the Federal Land Policy and 
M

In [47]:
# Loop through first 15 rows
for i in range(5):
    print(f"--- START PAGE {i} ---")
    print(pages.page_text[i])
    print(f"--- END PAGE {i} ---")
    print()  # Empty line for separation

--- START PAGE 0 ---
Coldfoot Cell Tower, F-97066 
 
Page 1 
 
Environmental Assessment 
GCI Coldfoot Cell Tower 
DOI-BLM-AK-2016-F030-0009-EA  
 
 
 
 
 
 
 
Bureau of Land Management 
Central Yukon Field Office 
1150 University Avenue 
Fairbanks, Alaska  99709 
 
February 2016 
 

--- END PAGE 0 ---

--- START PAGE 1 ---
Coldfoot Cell Tower, F-97066 
 
Page 2 
 
1. INTRODUCTION 
 
IDENTIFYING INFORMATION 
 
Title, EA Number, and Type of Project: 
 
DOI-BLM-AK-F030-2016-0009-EA; Communication Site 
 
Location of Proposed Action: 
 
The proposed tower center coordinates are 67° 15' 16.35" N, 150° 11' 05.99" W (NAD 83) near 
the Coldfoot Airport, mile post 175 off the Dalton Highway more particularly described as 
within Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, Alaska. 
 
Name and Location of Preparing Office: 
 
Central Yukon Field Office, 1150 University Avenue, Fairbanks, Alaska 99709 
 
Identify the case file number:  F-97066 
 
Applicant Name:  GCI Communication Corporation

In [48]:
pages

,document_id,page_number,page_text
0,2d9415b3c9309d9670f09501ca618ce3,1,"Coldfoot Cell Tower, F-97066 \n \nPage 1 \n \nEnvironmental Assessment \nGCI Coldfoot Cell Tower \nDOI-BLM-AK-2016-F030-0009-EA \n \n \n \n \n \n \n \nBureau of Land Management \nCentral Yukon Field Office \n1150 University Avenue \nFairbanks, Alaska 99709 \n \nFebruary 2016 \n \n"
1,2d9415b3c9309d9670f09501ca618ce3,2,"Coldfoot Cell Tower, F-97066 \n \nPage 2 \n \n1. INTRODUCTION \n \nIDENTIFYING INFORMATION \n \nTitle, EA Number, and Type of Project: \n \nDOI-BLM-AK-F030-2016-0009-EA; Communication Site \n \nLocation of Proposed Action: \n \nThe proposed tower center coordinates are 67° 15' 16.35"" N, 150° 11' 05.99"" W (NAD 83) near \nthe Coldfoot Airport, mile post 175 off the Dalton Highway more particularly described as \nwithin Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, Alaska. \n \nName and Loca..."
2,2d9415b3c9309d9670f09501ca618ce3,3,"Coldfoot Cell Tower, F-97066 \n \n \n \n \n \nPage 3 \n \n \n \nFigure 1. Proposed GCI Wireless facility location Aerial photograph of the Coldfoot airport, access road and general area.\n"
3,2d9415b3c9309d9670f09501ca618ce3,4,"Coldfoot Cell Tower, F-97066 \n \nPage 4 \n \n2. PROPOSED ACTION AND ALTERNATIVES \nDESCRIPTION OF THE PROPOSED ACTION \nGCI is proposing to install a communication tower site on public lands near the Coldfoot airport, \nmile post 175 off the Dalton Highway within Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, \ncontaining 2.29 acre. They propose a 20 year authorization with the option of renewal to \nconstruct, operate, maintain, and decommission. \n \nThe facility would provide UMTS (3G..."
4,2d9415b3c9309d9670f09501ca618ce3,5,"Coldfoot Cell Tower, F-97066 \n \nPage 5 \n \nDESCRIPTION OF ALTERNATIVES ANALYZED IN DETAIL \n \nNo Action Alternative \n \nUnder the No Action Alternative, BLM would not grant the lease. Currently there is no wireless \nvoice and data coverage in Coldfoot, and that would not change under the No Action Alternative. \n \nALTERNATIVES CONSIDERED BUT NOT ANALYZED IN DETAIL \n \nGCI considered the collocation of its wireless facilities on the Summit Telephone tower located \nwithin the communi..."
...,...,...,...
469101,506b827e66a462aa18e39c3eac695fda,245,"Dog River Pipeline Replacement | Environmental Assessment \n \n245 \n \nMarshal, D. B., M. G. Hunter, A. L.Contreras, Eds. 2003. Brids of Oregon: A General Reverence. \nOregon State University Press, Corvallis, OR. 768Pp. \nMartin, S.K. 1994. Feeding ecology of American martens and fishers. In: Buskirk, S.W.; Harestat, A.S.; \nRaphael, M.B., comps. eds. Martens, sables, and fishers: biology and conservation. Ithaca, NY: Cornell \nUniversity Press: 297-315. \nMcComb, B., Kellogg, L., and..."
469102,506b827e66a462aa18e39c3eac695fda,246,"Dog River Pipeline Replacement | Environmental Assessment \n \n246 \n \nThomas, J.W., Editor. 1979. Wildlife habitats in managed forests – The Blue Mountains of Oregon and \nWashington. U.S. Dep. Agric., For. Serv. Handb. 533. 512pp. \nThomas, J.W., R.G. Anderson, C. Maser, and E.L. Bull. 1979. Snags. Pages 60-77 in J.W. Thomas, \neditor. Wildlife habitats in managed forests: Agriculture handbook No. 553. USDA Forest Service, \nWashington, D.C. \nThomas, J.W., E.D. Forsman, J.B. Lint, E...."
469103,506b827e66a462aa18e39c3eac695fda,247,"Dog River Pipeline Replacement | Environmental Assessment \n \n247 \n \nBotany \nHarpel, J. and R. Helliwell. 2005. Conservation Assessment for Schistostega pennata (Hedw.) Web & \nMohr. USDA Forest Service and USDI Bureau of Land Management. \nHitchcock, C.L. and A. Cronquist. 1987. Flora of the Pacific Northwest. Seattle: University of \nWashington Press. \nInteragency Special Status/Sensitive Species Program (ISSSSP) species factsheets for bryophytes, \nlichens, and fungi. USDA Forest Ser..."
469104,506b827e66a462aa18e39c3eac695fda,248,"Dog River Pipeline Replacement | Environmental Assessment \n \n248 \n \nUSDA Forest Service, Region 6, Pacifi

In [49]:
docs_pages = documents.merge(pages, on='document_id', how='inner')
docs_pages.head()


,project_id,document_id,document_type,document_title,prepared_by,ce_category,file_id,file_name,total_pages,main_document,page_number,page_text
0,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,1,"Coldfoot Cell Tower, F-97066 \n \nPage 1 \n \nEnvironmental Assessment \nGCI Coldfoot Cell Tower \nDOI-BLM-AK-2016-F030-0009-EA \n \n \n \n \n \n \n \nBureau of Land Management \nCentral Yukon Field Office \n1150 University Avenue \nFairbanks, Alaska 99709 \n \nFebruary 2016 \n \n"
1,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,2,"Coldfoot Cell Tower, F-97066 \n \nPage 2 \n \n1. INTRODUCTION \n \nIDENTIFYING INFORMATION \n \nTitle, EA Number, and Type of Project: \n \nDOI-BLM-AK-F030-2016-0009-EA; Communication Site \n \nLocation of Proposed Action: \n \nThe proposed tower center coordinates are 67° 15' 16.35"" N, 150° 11' 05.99"" W (NAD 83) near \nthe Coldfoot Airport, mile post 175 off the Dalton Highway more particularly described as \nwithin Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, Alaska. \n \nName and Loca..."
2,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,3,"Coldfoot Cell Tower, F-97066 \n \n \n \n \n \nPage 3 \n \n \n \nFigure 1. Proposed GCI Wireless facility location Aerial photograph of the Coldfoot airport, access road and general area.\n"
3,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,4,"Coldfoot Cell Tower, F-97066 \n \nPage 4 \n \n2. PROPOSED ACTION AND ALTERNATIVES \nDESCRIPTION OF THE PROPOSED ACTION \nGCI is proposing to install a communication tower site on public lands near the Coldfoot airport, \nmile post 175 off the Dalton Highway within Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, \ncontaining 2.29 acre. They propose a 20 year authorization with the option of renewal to \nconstruct, operate, maintain, and decommission. \n \nThe facility would provide UMTS (3G..."
4,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,5,"Coldfoot Cell Tower, F-97066 \n \nPage 5 \n \nDESCRIPTION OF ALTERNATIVES ANALYZED IN DETAIL \n \nNo Action Alternative \n \nUnder the No Action Alternative, BLM would not grant the lease. Currently there is no wireless \nvoice and data coverage in Coldfoot, and that would not change under the No Action Alternative. \n \nALTERNATIVES CONSIDERED BUT NOT ANALYZED IN DETAIL \n \nGCI considered the collocation of its wireless facilities on the Summit Telephone tower located \nwithin the communi..."


In [50]:
docs_pages.document_type.unique()

array(['EA', '', 'OTHER', 'ROD', 'FONSI', 'DEA'], dtype=object)

In [51]:
test = docs_pages[0:100]
test
#test.shape

,project_id,document_id,document_type,document_title,prepared_by,ce_category,file_id,file_name,total_pages,main_document,page_number,page_text
0,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,1,"Coldfoot Cell Tower, F-97066 \n \nPage 1 \n \nEnvironmental Assessment \nGCI Coldfoot Cell Tower \nDOI-BLM-AK-2016-F030-0009-EA \n \n \n \n \n \n \n \nBureau of Land Management \nCentral Yukon Field Office \n1150 University Avenue \nFairbanks, Alaska 99709 \n \nFebruary 2016 \n \n"
1,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,2,"Coldfoot Cell Tower, F-97066 \n \nPage 2 \n \n1. INTRODUCTION \n \nIDENTIFYING INFORMATION \n \nTitle, EA Number, and Type of Project: \n \nDOI-BLM-AK-F030-2016-0009-EA; Communication Site \n \nLocation of Proposed Action: \n \nThe proposed tower center coordinates are 67° 15' 16.35"" N, 150° 11' 05.99"" W (NAD 83) near \nthe Coldfoot Airport, mile post 175 off the Dalton Highway more particularly described as \nwithin Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, Alaska. \n \nName and Loca..."
2,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,3,"Coldfoot Cell Tower, F-97066 \n \n \n \n \n \nPage 3 \n \n \n \nFigure 1. Proposed GCI Wireless facility location Aerial photograph of the Coldfoot airport, access road and general area.\n"
3,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,4,"Coldfoot Cell Tower, F-97066 \n \nPage 4 \n \n2. PROPOSED ACTION AND ALTERNATIVES \nDESCRIPTION OF THE PROPOSED ACTION \nGCI is proposing to install a communication tower site on public lands near the Coldfoot airport, \nmile post 175 off the Dalton Highway within Sec. 16, T. 28 N., R. 12 W., Fairbanks Meridian, \ncontaining 2.29 acre. They propose a 20 year authorization with the option of renewal to \nconstruct, operate, maintain, and decommission. \n \nThe facility would provide UMTS (3G..."
4,{'value': '6e9fc6608e30977c74305d2a98628a13'},2d9415b3c9309d9670f09501ca618ce3,EA,,[],,2d9415b3c9309d9670f09501ca618ce3,EA_PDF.pdf,12,YES,5,"Coldfoot Cell Tower, F-97066 \n \nPage 5 \n \nDESCRIPTION OF ALTERNATIVES ANALYZED IN DETAIL \n \nNo Action Alternative \n \nUnder the No Action Alternative, BLM would not grant the lease. Currently there is no wireless \nvoice and data coverage in Coldfoot, and that would not change under the No Action Alternative. \n \nALTERNATIVES CONSIDERED BUT NOT ANALYZED IN DETAIL \n \nGCI considered the collocation of its wireless facilities on the Summit Telephone tower located \nwithin the communi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,{'value': 'e36ab077e8323fbb6f67fb76b47c5820'},162d938ba342db581c995e133498c49b,,,[],,162d938ba342db581c995e133498c49b,AppendixC_Maps&Figs.pdf,3,NO,3,\n \n
96,{'value': 'e36ab077e8323fbb6f67fb76b47c5820'},8b2826e23ec69ff9e05f6294c91a3d9b,,,[],,8b2826e23ec69ff9e05f6294c91a3d9b,AppendixD_BE.pdf,37,NO,1,\n \n \n \n \nAPPENDIX D – BIOLOGICAL EVALUATION \n
97,{'value': 'e36ab077e8323fbb6f67fb76b47c5820'},8b2826e23ec69ff9e05f6294c91a3d9b,,,[],,8b2826e23ec69ff9e05f6294c91a3d9b,AppendixD_BE.pdf,37,NO,2,"\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nT\n MATER\nB\nRIAL\nBIOL \nFIS\nL SO\nOGI \nSHER\nURC\nCAL\nR EH\nCE E \nL EVA\n2\nHRE \nEXPA\nALU\n202\nPREPA\nHIMES CON\n3301 WEST\nCHANDLE \n(480) 8\nwww.himes‐\nAu\nNBE\nANSI\nUATI\n22 \nARED BY: \nNSULTING LLC\n GENOA WAY\nR, AZ 85226 \n899‐5708 \nconsulting.co\nugust \nERG \nION \nION \nC \nY \nom \n"
98,{'value': 'e36ab077e8323fbb6f67fb76b47c5820'},8b2826e23ec69ff9e05f6294c91a3d9b,,,[],,8b2826e23ec69ff9e05f6294c91a3d9b,AppendixD_BE.pdf,37,NO,3,"\n \n \n \n \n \n \nTABLE OF CONTENTS \n \nSection\n \n \n \n \n \n \n \n \n \n \n \nPage\n \n \nPROJE

In [52]:
import spacy

from scattertext import SampleCorpora, produce_scattertext_explorer
from scattertext import produce_scattertext_html
from scattertext.CorpusFromPandas import CorpusFromPandas

nlp = spacy.load('en_core_web_sm')
#convention_df = SampleCorpora.ConventionData2012.get_data()
short_docs = docs_pages[0:100]
corpus = CorpusFromPandas(short_docs,
                          category_col='document_type',
                          text_col='page_text',
                          nlp=nlp).build()


# create html document
html = st.produce_scattertext_explorer(corpus,
                                       category='EA',
                                       category_name='Environmental Assessment',
                                       not_category_name='Other',
                                       width_in_pixels=1000,
                                       minimum_term_frequency=5,
                                       metadata=short_docs['document_id'])

open('../output/scattertext/ea.html', 'wb').write(html.encode('utf-8'))
print('Open ../output/scattertext/ea.html in web browser.')

Open ../output/scattertext/ea.html in web browser.
